In [ ]:
import geopandas as gpd
import os
import warnings

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Input file path
roads_path = os.path.join(base_dir, "street_network.shp")

# Output file path
output_path = os.path.join(base_dir, "street_network.shp")

# Load shapefile
roads = gpd.read_file(roads_path)

In [2]:
def count_ramps(roads, highway_name, on_ramp_keywords, off_ramp_keywords):
    """
    Counts the number of on-ramps and off-ramps for a specified highway.
    """
    # Count instances for on-ramps
    on_ramps_count = roads['STREETNAME'].str.contains('|'.join(on_ramp_keywords), na=False).sum()

    # Count instances for off-ramps
    off_ramps_count = roads['STREETNAME'].str.contains('|'.join(off_ramp_keywords), na=False).sum()

    # Print the results
    print(f"{highway_name} Ramps:")
    print(f"Number of on-ramps: {on_ramps_count}")
    print(f"Number of off-ramps: {off_ramps_count}")
    
    return on_ramps_count, off_ramps_count

In [ ]:
# Define I-90 ramp keywords
off_ramps_90 = ["RAMP-RTS 90", "RAMP-RT 90"]
on_ramps_90 = ["TO RT 90", "TO RTS 90"]

# Count ramps for I-90
on_count_90, off_count_90 = count_ramps(roads, "I-90", on_ramps_90, off_ramps_90)

I-90 Ramps:
Number of on-ramps: 213
Number of off-ramps: 174


In [4]:
# Define I-93 ramp keywords
off_ramps_93 = ["RAMP-RTS 93", "RAMP-RT 93"]
on_ramps_93 = ["TO RT 93", "TO RTS 93"]

# Count ramps for I-93
on_count_93, off_count_93 = count_ramps(roads, "I-93", on_ramps_93, off_ramps_93)

I-93 Ramps:
Number of on-ramps: 279
Number of off-ramps: 269


In [5]:
def update_ramps(roads, highway_name, on_ramp_keywords, off_ramp_keywords):
    """
    Updates the qNoAccess attribute for on-ramps and off-ramps for a specified highway.
    On-ramps are assigned 98, and off-ramps are assigned 99.
    """
    # Update qNoAccess attribute for on-ramps
    roads.loc[roads['STREETNAME'].str.contains('|'.join(on_ramp_keywords), na=False), 'qNoAccess'] = 98

    # Update qNoAccess attribute for off-ramps
    roads.loc[roads['STREETNAME'].str.contains('|'.join(off_ramp_keywords), na=False), 'qNoAccess'] = 99

    print(f"{highway_name} ramps updated: On-ramps set to 98, Off-ramps set to 99.")
    return roads

# Update ramps for I-90
roads = update_ramps(roads, "I-90", on_ramps_90, off_ramps_90)

# Update ramps for I-93
roads = update_ramps(roads, "I-93", on_ramps_93, off_ramps_93)

I-90 ramps updated: On-ramps set to 98, Off-ramps set to 99.
I-93 ramps updated: On-ramps set to 98, Off-ramps set to 99.


In [ ]:
from datetime import datetime

# Define the condition
condition = ((roads['qNoAccess'] == 98) | (roads['qNoAccess'] == 99)) & (roads['qExclude'] == 1)

# Update qExclude to 0 using the condition
roads.loc[condition, 'qExclude'] = 0

# Save the updated shapefile
roads.to_file(output_path)

In [ ]:
# Prepare the current date
current_date = datetime.now().strftime("%m/%d/%Y")

# Append the edit comment appropriately
roads.loc[condition, 'qEditCmnts'] = roads.loc[condition, 'qEditCmnts'].apply(
    lambda cmnt: (
        cmnt + f"; ND changed qExclude from 1 to 0 on {current_date}"
        if cmnt else
        f"ND changed qExclude from 1 to 0 on {current_date}"
    )
)

In [7]:
# Display value counts for the 'qNoAccess' column
qNoAccess_counts = roads['qNoAccess'].value_counts()
print(qNoAccess_counts)

qNoAccess
0     34911
1      1152
99      371
98      362
Name: count, dtype: int64
